<a href="https://colab.research.google.com/github/lab30041954/ML_Course/blob/main/Assignments/Assignment%201%20-%20The%20prof%20solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1 - The prof solution

## Question 1

### Task

In this data set, there is a mix of scales, which can be visualized with the method `.describe()`. This mix slows down the convergence in the method `.fit()`, so we had to increase the parameter `max_iter`. This is not a relevant issue in a model as simple as the one used in this example, but it will be in the complex models that will appear later in this course, such as the neural network models. Try different values for `max_iter` in the specification of `LogisticRegression()` (input 7) and examine how the maximum number of iterations affects the model accuracy in this case.

### Preparation

We start by creating the features matrix `X` and the target vector `y`, as usual in supervised learning. The following code chunk has already been used in class.

In [1]:
import pandas as pd
path = 'https://raw.githubusercontent.com/lab30041954/Data/main/'
df = pd.read_csv(path + 'churn.csv', index_col=0)
y = df['churn']
X = df.drop(columns='churn')

### Visualizing the mix of scales

As mentioned in the assignment introduction, the method `.describe()` extracts a stat summary that helps to visualize the mix of scales in the training data.

In [2]:
df.describe()

,aclength,intplan,dataplan,ommin,omcall,otmin,otcall,ngmin,ngcall,imin,icall,cuscall,churn
count,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,100.900800,0.104600,0.310200,180.11282,70.943800,194.784180,93.534600,20.054680,9.176200,10.475080,2.965800,1.514000,0.193600
std,39.800761,0.306068,0.462622,55.33402,18.496439,47.706068,22.173839,14.061867,4.649331,6.406954,2.190979,1.240045,0.395159
min,0.000000,0.000000,0.000000,12.70000,8.000000,31.100000,21.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,73.000000,0.000000,0.000000,142.97500,59.000000,161.900000,79.000000,9.800000,6.000000,6.000000,1.000000,1.000000,0.000000
50%,100.000000,0.000000,0.000000,179.50000,71.000000,195.350000,94.000000,16.900000,9.000000,10.500000,3.000000,1.000000,0.000000
75%,127.000000,0.000000,1.000000,217.62500,84.000000,227.600000,109.000000,27.200000,12.000000,14.600000,4.000000,2.000000,0.000000
max,227.000000,1.000000,1.000000,408.00000,145.000000,364.100000,179.000000,114.200000,30.000000,45.200000,16.000000,8.000000,1.000000


### Default specification

We start with the **default** specification of the logistic regression model, which has `max_iter=100`. We call this model `clf1`, so we can distinguish it from the alternative models trained later.

In [3]:
from sklearn.linear_model import LogisticRegression
clf1 = LogisticRegression()

In [4]:
clf1.fit(X , y)

/opt/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


We get a warning suggesting a higher allowance for the number of iterations. This is aligned with the fact that the accuracy is lower than that obtained on class.

In [5]:
round(clf1.score(X, y), 3)

0.814

### Increase the number of iterations

We increase the maximum number of iteratrions to 500 (this number is arbitrary).

In [6]:
clf2 = LogisticRegression(max_iter=500)
clf2.fit(X, y)

/opt/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 500 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=500).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,500
,multi_class,'deprecated'


In [7]:
clf2.score(X, y)

0.841

Right now, the accuracy is about the same as that obtained in class, although we didn't achieve the convergence.

### Increase a bit more

Raising the `max_iter` parameter value to 1,000 seems to settle the issue.

In [8]:
clf3 = LogisticRegression(max_iter=1000)
clf3.fit(X, y)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [9]:
clf3.score(X, y)

0.8416

Note that, not getting the warning with this specification means that convergence was achieved with less than 1,000 iterations. Increasing the the `max_iter` parameter value even more will not change the effective number of iterations needed, and the process will stop at the same point.

## Question 2

### Task

Rescale all the features which are not dummies and train the logistic regression classifier with the default number of iterations. Do you get a warning about non-convergence now?

### Rescaling

The ideal scale is that in which the range of values of every feature is the interval 0-1. Ideal means here that (a) convergence is potentially faster and (b) the resulting model has a potentially better performance. The transformation uses a linear formula:
$$y = \fraction{x - min(x)}{max(x) - min(x)}.

This is called **min-max normalization**. It can be applied directly, replacing every feature by its rescaled version, as

In [10]:
df['aclength'] = (df['aclength'] - df['aclength'].min())/(df['aclength'].max() - df['aclength'].min())

We can check the outcome easily:

In [11]:
df['aclength'].describe()

count    5000.000000
mean        0.444497
std         0.175334
min         0.000000
25%         0.321586
50%         0.440529
75%         0.559471
max         1.000000
Name: aclength, dtype: float64

With a bit of copypasting, we apply this to the rest of the features.

In [12]:
df['ommin'] = (df['ommin'] - df['ommin'].min())/(df['ommin'].max() - df['ommin'].min())
df['omcall'] = (df['omcall'] - df['omcall'].min())/(df['omcall'].max() - df['omcall'].min())
df['otmin'] = (df['otmin'] - df['otmin'].min())/(df['otmin'].max() - df['otmin'].min())
df['otcall'] = (df['otcall'] - df['otcall'].min())/(df['otcall'].max() - df['otcall'].min())
df['ngmin'] = (df['ngmin'] - df['ngmin'].min())/(df['ngmin'].max() - df['ngmin'].min())
df['ngcall'] = (df['ngcall'] - df['ngcall'].min())/(df['ngcall'].max() - df['ngcall'].min())
df['imin'] = (df['imin'] - df['imin'].min())/(df['imin'].max() - df['imin'].min())
df['icall'] = (df['icall'] - df['icall'].min())/(df['icall'].max() - df['icall'].min())
df['cuscall'] = (df['cuscall'] - df['cuscall'].min())/(df['cuscall'].max() - df['cuscall'].min())

In [13]:
df.describe()

,aclength,intplan,dataplan,ommin,omcall,otmin,otcall,ngmin,ngcall,imin,icall,cuscall,churn
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.444497,0.104600,0.310200,0.423508,0.459444,0.491544,0.459080,0.175610,0.305873,0.231750,0.185363,0.189250,0.193600
std,0.175334,0.306068,0.462622,0.139980,0.135011,0.143261,0.140341,0.123134,0.154978,0.141747,0.136936,0.155006,0.395159
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.321586,0.000000,0.000000,0.329560,0.372263,0.392793,0.367089,0.085814,0.200000,0.132743,0.062500,0.125000,0.000000
50%,0.440529,0.000000,0.000000,0.421958,0.459854,0.493243,0.462025,0.147986,0.300000,0.232301,0.187500,0.125000,0.000000
75%,0.559471,0.000000,1.000000,0.518404,0.554745,0.590090,0.556962,0.238179,0.400000,0.323009,0.250000,0.250000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Of course, this is not elegant. Professionals never repeat things explicitly, they use loops. A programmer will (a) list the features, (b) create a function which does the job, and (c) loop over the list. The code chunk could be:

In [14]:
numeric_cols = ['aclength', 'ommin', 'omcall', 'otmin', 'otcall', 'ngmin', 'ngcall', 'imin', 'icall', 'cuscall']
def f(x):
    y = (x - x.min())/(x.max() - x.min())
    return y
for x in numeric_cols:
    df[x] = f(df[x])

scikit-learn has a class for this, named `MinMaxScaler`.

### New features matrix

The new features matrix will updated by executing again the command that was used to create the first version.  

In [15]:
X = df.drop(columns='churn')

### Retraining the model with the rescaled data

Now we don't need extra iterations.

In [16]:
clf = LogisticRegression()
clf.fit(X, y)
clf.score(X, y)

0.841